First things first.  Load the package.

In [1]:
Sys.setenv(R_CONFIG_ACTIVE = "jupyter")
suppressMessages({
  devtools::load_all()
  data(pcd)
})

## cli / logger integration

As code shifts from prototype to production, logging becomes important.  Given the ubiquity of the [cli package](https://cli.r-lib.org/reference/index.html), it would be nice if messages that were originally headed for an interactive terminal were seamlessly redirected into the logs.

We utilize the default_handler machinery available in the cli package, and this option is set when metayer is loaded.  So we must unset the option in order to observe the default behavior:

### vanilla cli

We utilize the default handler machinery available in the cli package, and this is put in place when metayer is loaded.  To recover the usual behavior, the `cli.default_handler` option must be cleared.

In [2]:
# default behavior of cli_alert_warning
withr::with_options(
  list(cli.default_handler = NULL), 
  cli::cli_alert_warning("a message to our listeners")          
)

! a message to our listeners



### setting cli.default_handler

With the metayer handler set, we obtain something similar, but now with an attempt at delivering contextual information.  In particular, Jupyter returns a log record:  "[INFO-unknown]", followed by the message.

In [3]:
cli::cli_alert_danger("a whisper of danger for our viewers")

[INFO/unknown] x a whisper of danger for our viewers


We can show that our machinery also persists messages in the logs.

In [4]:
# create a read/write context for a logfile
tmp = tempfile()
invisible(
  with_wrapped_logger(
    {
      suppressMessages(
        cli::cli_alert_warning("an understated warning for our members")
      )
    },
    logfile = tmp,
    namespace = "unknown"
  )
)

cat(
  xfun::read_utf8(tmp)
)

[INFO/unknown] ! an understated warning for our members

### enriching the context

In the above examples, "[INFO-unknown]" doesn't add much.  However, the metayer versions of these functions do:

In [5]:
cli_alert_warning("a proper warning for our VIPs")

[WARN/global.cli] ! a proper warning for our VIPs


Now we see that the alert warning produces a log record with a WARN level.  We also observe an environment context, "global.cli".  One can contrast the "global" environment with a direct call into logger:

In [6]:
log_warn("a proper warning for our VIPs on another channel")

[WARN/global] a proper warning for our VIPs on another channel


If the log record originates from a cli call, it will have the ".cli" suffix.  Otherwise, it will carry the name of the namespace in which it was defined.  A contrived example below shows the pattern, e.g. "metayer.cli":

In [7]:
foo <- function() cli_alert_danger("a cry for help to our fearless leaders")
environment(foo) <- asNamespace("metayer")
foo()

[ERROR/metayer.cli] x a cry for help to our fearless leaders


## glue semantics

Of course, these wrapped functions are designed to implement the glue semantics that the usual cli functions enjoy.

In [8]:
v1 = "foo"
v2 = NULL
v3 = "bar"

cli_alert_info("{v1} {v2} {v3}")

[INFO/global.cli] i foo <null> bar


There is one notable change, and that is that we introduce "<null>" in place of NULL values.  This behavior can be adjusted with the `mty.cli_null` option defined in the config.yml file.